In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from umap import UMAP
from Bio import SeqIO
import bioframe
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import h5py
os.environ['CUDA_VISIBLE_DEVICES']='2'
from transformers import AutoModel,AutoTokenizer,AutoModelForMaskedLM
from tqdm import tqdm
import torch
import gpn.model
import sys
sys.path.append('../')
import utils

#model = AutoModel.from_pretrained("/home/ztang/multitask_RNA/model/GPN_finetune/checkpoint-2000000").to('cuda')
#tokenizer = AutoTokenizer.from_pretrained("/home/ztang/multitask_RNA/model/GPN_finetune/checkpoint-2000000")

# model = AutoModel.from_pretrained("gonzalobenegas/gpn-arabidopsis").to('cuda')
# tokenizer = AutoTokenizer.from_pretrained("gonzalobenegas/gpn-arabidopsis")

model = AutoModel.from_pretrained("/home/ztang/multitask_RNA/model/GPN_human/checkpoint-2000000").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("/home/ztang/multitask_RNA/model/GPN_human/checkpoint-2000000")

model.eval();
file = h5py.File('../../data/rna_stable/insert_dataset.h5','r')

/home/ztang/.conda/envs/gpn_env/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ztang/.conda/envs/gpn_env/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/ztang/.conda/envs/gpn_env/lib/python3.9/site-packages/umap/distances.py:

In [2]:
gpn_output = h5py.File('../../data/rna_stable/gpn_human_embed.h5','w')
batch_size = 32
for dataset in ['test','train','valid']:
    key = 'X_'+dataset
    onehot = file[key]
    string_seq = utils.onehot_to_seq(onehot)

    token_seq = tokenizer.batch_encode_plus(string_seq, max_length=512,padding = 'max_length')
    output_cache = []
    for seq_i in tqdm(range(0,len(token_seq['input_ids']),batch_size)):
        seq_batch = torch.tensor(token_seq['input_ids'][seq_i:seq_i+batch_size]).to('cuda')
        output_seq = model(seq_batch).last_hidden_state.cpu().detach().numpy()
        output_cache.extend(output_seq[:,:173,:])
    gpn_output.create_dataset(name=key,data = np.array(output_cache))
    gpn_output.create_dataset(name='Y_'+dataset,data = file['Y_'+dataset][:])

  0%|          | 0/36 [00:00<?, ?it/s]/home/ztang/.conda/envs/gpn_env/lib/python3.9/site-packages/torch/nn/modules/conv.py:309: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,
100%|██████████| 36/36 [00:02<00:00, 15.69it/s]


In [3]:
gpn_output.close()

In [4]:
np.array(output_cache).shape

(1149, 173, 512)

## Reproducibility check

In [5]:
import h5py 
import numpy as np

In [6]:
citra_f = h5py.File('/home/ztang/multitask_RNA/data/rna_stable/citra_copy/gpn_human_embed.h5','r')
local_f = h5py.File('/home/ztang/multitask_RNA/data/rna_stable/gpn_human_embed.h5','r')